In [1]:
# Columns to add to the data:

# posts
# total wikipedia mentions
# total mentions in URLs
# total replies
# total reply score

In [ ]:
# new tables - post URLs, comment urls

# columns: post_id, created_at, updated_at, raw url, parsed url, domain, parsed title, processed title, article IDs???, 
# columns: comment_id, post_id, parent_id, created_at, last_modfied_at, raw url, parsed url, domain, article, redirecttitel, article IDs???

In [2]:
import pandas as pd
article_dates_unique = pd.read_hdf('data/article_dates_unique.h5', 'df')
for lang in article_dates_unique['lang'].unique():
    print(lang)

en
ru
pt
it
ro
nl
es
fr
no
de
simple
eu
ja
zh
tr
hi
sl
ar
cs
csb
mk
szl
hr
uk
sr
sk
bg
cu
dsb
bs
hsb
sh
pl
rue
be
ml
el
fa
sw
ko
id
www
hu
fi
vi
eo
th
sv
ta
lt
sco
he
ga
zh-classical
hy
da
ur
ast
ca
als
et
nan
tl
arz
ka
az
sq
is
la
ms
got
fy
stq
lb
sa
rmy
sd
bar
af
nds
vo
ang
yi
pcd
cy
kw
lv
ltg
pap
so
vec
qu
nap
si
my
nds-nl
zh-yue
nostalgia
ie
tk
bn
nah
lmo
nn
ku
mr
ba
kk
zh-min-nan
gl
avk
vep
ary
ia
hif
ps
skr
pnb
ks
bpy
pa
or
te
kn
gu
jam
ceb
mn
shi
kab
azb
uz
ky
mni
ne
wuu
nv
pam
cbk-zam
bh
tg
am
be-tarask
ty
vls
co
br
frp
nrm
gcr
oc
mwl
wa
li
pfl
gd
rm
fur
pdc
tpi
ksh
fo
os
an
pi
se
bat-smg
gom
olo
gn
mg
mt
arc
test
war
roa-rup
crh
ace
tyv
gv
eml
scn
pnt
zea
lad
tt
sc
jbo
km
gan
ha
frr
ckb
pih
lfn
yo
bo
min
nov
io
ht
tn
pms
kv
ab
aa
bcl
thankyou
ug
sat
nqo
kl
xh
jv
om
zu
bi
ff
haw
lij
mai
ext
wo
tay
trv
cdo
hak
dv
ss
sm
rw
cv
chy
mi
cr
xmf
su
bug
hyw
bjn
roa-tara
kaa
diq
atj
ban
fiu-vro
na
as
ady
bxr
to
chr
glk
mdf
ay
ce
mrj
lbe
udm
myv
mhr
krc
koi
szy
gag
lez
sah
za
smn
gur
pcm


In [1]:
import markdown
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urlunparse
import re
import pandas as pd
import requests
import asyncio
import aiohttp
from unicodedata import category
import time
import pickle
import os

# Assume 'posts' DataFrame is already loaded, containing a 'body' column.
# posts = pd.read_hdf('data/posts.h5').reset_index(drop=True)

HEADERS = {'user-agent': 'wikireddit p.gildersleve@exeter.ac.uk'}
MAX_CONCURRENCY = 100

if os.path.exists('.temp/rd_url_cache_p.pkl'):
    with open('.temp/rd_url_cache_p.pkl', 'rb') as f:
        RD_URL_CACHE_P = pickle.load(f)
else:
    RD_URL_CACHE_P = {True: {}, False: {}}
    
if os.path.exists('.temp/rd_url_cache_tp.pkl'):
    with open('.temp/rd_url_cache_tp.pkl', 'rb') as f:
        RD_URL_CACHE_TP = pickle.load(f)
else:
    RD_URL_CACHE_TP = {}

# --- Helper Functions ---

def clean_wikipedia_domain(link):
    if 'wikipedia.org' not in link:
        return None
    try:
        parsed = urlparse(link)
    except ValueError as ex:
        print(ex)
        return None
    domain = parsed.netloc.lower()
    while domain and category(domain[-1])[0]=='P':
        domain = domain[:-1]
    # get subdomain
    # subdomain = domain.split('wikipedia.org')[0]
    # if (subdomain not in subdomains) and subdomain:
    #     return None
    # Domain must be exactly 'wikipedia.org' or end with '.wikipedia.org'
    if domain == 'wikipedia.org' or domain.endswith('.wikipedia.org'):
        return urlunparse(('https', domain, parsed.path, parsed.params, parsed.query, parsed.fragment))
    else:
        # print(link, 'not wikipedia')
        return None

URL_REGEX = re.compile(
    r'(https?://[^\s<>\[\]{}|]+(?:wikipedia\.org)[^\s<>\[\]{}|]*)|((?:[a-z0-9-]+\.)*wikipedia\.org[^\s<>\[\]{}|]*)',
    re.IGNORECASE
)

def extract_plain_links(text):
    plain_links = []
    for match in URL_REGEX.finditer(text):
        candidate = match.group(1) if match.group(1) else match.group(2)
        plain_links.append(candidate.strip())
    return plain_links

def normalize_links(links):
    normalized = []
    for link in links:
        if not re.match(r'^https?://', link, re.IGNORECASE):
            link = 'https://' + link
        normalized.append(link)
    return normalized

def filter_wikipedia_links(links):
    wiki_links = set()
    for link in links:
        cleanlink = clean_wikipedia_domain(link)
        if cleanlink:
            wiki_links.add(cleanlink)
    return wiki_links

def extract_links_from_text(text):
    # Convert Markdown to HTML and extract markdown-parsed URLs
    html = markdown.markdown(text, extensions=['extra'])
    soup = BeautifulSoup(html, 'html.parser')
    a_tags = soup.find_all('a', href=True)
    markdown_links = [a['href'].strip() for a in a_tags]

    # Remove these <a> tags to avoid double counting
    for a_tag in a_tags:
        a_tag.decompose()
    cleaned_text = soup.get_text()

    # Extract plain links
    plain_links = extract_plain_links(cleaned_text)

    # Combine, normalize, and deduplicate
    all_links = set(normalize_links(markdown_links + plain_links))

    # Filter to Wikipedia links
    wiki_links = filter_wikipedia_links(all_links)
    return list(wiki_links)

def reextract_links(text):
    plain_links = extract_plain_links(text)
    all_links = set(normalize_links(plain_links))
    wiki_links = filter_wikipedia_links(all_links)
    return list(wiki_links)

# --- Async Validation Functions ---
async def async_validate_link(session, url, timeout=5, retries=10, allow_redirects=False):
    try:
        async with session.head(url, allow_redirects=allow_redirects, timeout=timeout) as r:
            if 200 <= r.status < 400:
                return True, r.status, str(r.url)
            elif r.status == 429 and retries > 0:
                await asyncio.sleep(1)  # Wait for 1 second before retrying
                return await async_validate_link(session, url, timeout, retries - 1, allow_redirects=allow_redirects)
            else:
                async with session.get(url, allow_redirects=allow_redirects, timeout=timeout) as r2:
                    if 200 <= r2.status < 400:
                        return True, r2.status, str(r2.url)
                    else:
                        return False, r2.status, str(r2.url)
    except Exception as ex:
        print(ex)
    return False, -1, None

async def async_validate_url_with_punctuation(session, url, timeout=5, retries=10, allow_redirects=False):
    is_valid, status, processed_url = await async_validate_link(session, url, timeout, retries, allow_redirects)
    if is_valid:
        return is_valid, status, processed_url

    # Try removing trailing punctuation
    while url and category(url[-1])[0]=='P':
        url = url[:-1]
        is_valid, status, processed_url = await async_validate_link(session, url, timeout, retries, allow_redirects)
        if is_valid:
            return is_valid, status, processed_url

    return False, status, processed_url

async def async_validate_url_with_textpunctuation(session, url, timeout=5):
    is_valid, status, processed_url = await async_validate_link(session, url, timeout)
    if is_valid:
        return is_valid, status, processed_url

    # Remove trailing alphanumeric and punctuation until punctuation is hit
    original_processed_url = processed_url
    # print('av', url)
    url = url.rstrip('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789:\'"_')
    if url[-5:] != 'wiki/' and url[-5:] != '.org/':
        is_valid, status, processed_url = await async_validate_url_with_punctuation(session, url, timeout=5)
        if is_valid:
            return is_valid, status, processed_url

    return False, status, original_processed_url

async def validate_urls_main(urls, retries=10, allow_redirects=False, max_concurrency=MAX_CONCURRENCY):
    semaphore = asyncio.Semaphore(max_concurrency)
    async with aiohttp.ClientSession(headers=HEADERS) as session:
        async def validate(url):
            if url in RD_URL_CACHE_P[allow_redirects]:
                return url, RD_URL_CACHE_P[allow_redirects][url]  # Return cached result
            
            # ṬODO, add str vaidatopm here for domain

            async with semaphore:
                result = await async_validate_url_with_punctuation(session, url,
                                                                      retries=retries,
                                                                      allow_redirects=allow_redirects)
                if result[1] != 429:
                    RD_URL_CACHE_P[allow_redirects][url] = result  # Cache result
                return url, result
        tasks = [validate(u) for u in urls]
        results = await asyncio.gather(*tasks)
        return results

async def validate_urls_main2(urls, max_concurrency=MAX_CONCURRENCY):
    semaphore = asyncio.Semaphore(max_concurrency)
    async with aiohttp.ClientSession(headers=HEADERS) as session:
        async def validate(url):
            if url in RD_URL_CACHE_TP:
                return url, RD_URL_CACHE_TP[url]  # Return cached result
            async with semaphore:
                result = await async_validate_url_with_textpunctuation(session, url)
                if result[1] != 429:
                    RD_URL_CACHE_TP[url] = result  # Cache result
                return url, result
        tasks = [validate(u) for u in urls]
        results = await asyncio.gather(*tasks)
        return results

# --- Processing ---

async def get_links_df(posts_df, column='body'):

    # print(len(posts_df))
    posts_df['extracted_links'] = posts_df[column].dropna().apply(extract_links_from_text)
    # print(posts_df['extracted_links'].iloc[0])
    # print(posts_df['extracted_links'].iloc[-1])
    links_df = posts_df.explode('extracted_links').rename(columns={'extracted_links':'extracted_url'}).reset_index(drop=True)
    links_df = links_df[~links_df['extracted_url'].isna()].drop_duplicates().copy()
    # print(len(links_df))
    # print('egl', links_df)

    return await process_links(links_df)

async def process_links(links_df, retry_count=0, max_retries=5, backoff_factor=3):
    chop_index = list(links_df.columns).index('extracted_url') + 1
    # 1) Initial validation
    urls = links_df['extracted_url'].unique()
    # print(urls)
    results = await validate_urls_main(urls)
    # print(len(results))
    df_results = pd.DataFrame([[x[0], *x[1]] for x in results],
                              columns=['extracted_url', 'valid_1', 'status_1', 'processed_url_1']).drop_duplicates('extracted_url')
    # print(len(df_results))
    # links_df.to_hdf('data/links_df_test.h5', key='df', mode='w')
    # df_results.to_hdf('data/df_results_test.h5', key='df', mode='w')
    links_df = links_df.merge(df_results, on='extracted_url', how='left').drop_duplicates()
    # links_df.to_hdf('data/links_df_test1.h5', key='df', mode='w')
    # print(len(links_df))
    # 2) Reextract for invalid URLs
    invalid_urls = links_df.loc[links_df['valid_1']==False, 'extracted_url'].unique()

    if len(invalid_urls) > 0:
        # Create a mapping DataFrame of extracted_url -> reextracted_url
        invalid_urls_series = pd.Series(invalid_urls, name='extracted_url')
        reextracted_list = []
        for u in invalid_urls_series:
            reex_urls = reextract_links(u)
            for rurl in reex_urls:
                reextracted_list.append((u, rurl))

        if reextracted_list:
            reextracted_df = pd.DataFrame(reextracted_list, columns=['extracted_url', 'reextracted_url']).drop_duplicates('extracted_url')
            # reextracted_df.to_hdf('data/reextracted_df_test.h5', key='df', mode='w')
            # print('re', reextracted_df)
            # Validate reextracted URLs
            re_urls = reextracted_df['reextracted_url'].unique()
            # print('ru', re_urls)
            results2 = await validate_urls_main(re_urls)
            df_results2 = pd.DataFrame([[x[0], *x[1]] for x in results2],
                                       columns=['reextracted_url', 'valid_2', 'status_2', 'processed_url_2']).drop_duplicates('reextracted_url')
            # print('r2', df_results2)
            # Merge second attempt results into reextracted_df
            # df_results2.to_hdf('data/df_results2_test.h5', key='df', mode='w')
            reextracted_df = reextracted_df.merge(df_results2, on='reextracted_url', how='left').drop_duplicates()
            # print('re2', reextracted_df)
            # Now merge reextracted results back to links_df using 'extracted_url'
            # prin t('l0', links_df)
            # reextracted_df.to_hdf('data/reextracted_df_test2.h5', key='df', mode='w')
            links_df = links_df.merge(reextracted_df, on='extracted_url', how='left', suffixes=('', '_second')).drop_duplicates()
            # print('lr', links_df)
            # Check still invalid after reextraction
            # Consider invalid if original attempt and reattempt still fail
            # links_df.to_hdf('data/links_df_test2.h5', key='df', mode='w')
            # print('ldf', len(links_df))
            # print('re', len(re_urls))
            # print('r2', len(results2))
            # print('rdf', len(reextracted_df))
            still_invalid_df = links_df.loc[
                (links_df['valid_1']==False) &
                ((links_df['valid_2'].isna()) | (links_df['valid_2']==False))
            ][['reextracted_url']].drop_duplicates().copy()
            # print('si', still_invalid_df)
            if len(still_invalid_df) > 0:
                # Validate still invalid URLs with textpunctuation
                # We need to look up the reextracted URLs again for these still invalid URLs
                # or possibly re-run the logic if reextraction is required again.
                # For now, let's assume we can directly use 'reextracted_df' to get them.

                # Filter reextracted_df for those matching still_invalid original URLs
                re_urls_third = still_invalid_df['reextracted_url'].unique()
                # print('r3', re_urls_third)
                results3 = await validate_urls_main2(re_urls_third)
                df_results3 = pd.DataFrame([[x[0], *x[1]] for x in results3],
                                           columns=['reextracted_url', 'valid_3', 'status_3', 'processed_url_3']).drop_duplicates('reextracted_url')

                # Merge third attempt results
                still_invalid_df = still_invalid_df.merge(df_results3, on='reextracted_url', how='left').drop_duplicates()
                
                # Merge back into links_df by original_url
                links_df = links_df.merge(still_invalid_df, on='reextracted_url', how='left', suffixes=('', '_third')).drop_duplicates()

    # Now links_df contains:
    # - original extracted_url from the posts
    # - initial validation results (valid, status, processed_url)
    # - second attempt validation results (valid_re, status_re, processed_url_re) via reextraction
    # - third attempt validation results (valid_third, status_third, processed_url_third)

    # You can now decide on final validity and merge results back into posts if needed.

    # add cols to links_df, if not present
    cols = ['extracted_url', 'valid_1', 'status_1',
       'processed_url_1', 'reextracted_url', 'valid_2', 'status_2', 'processed_url_2',
       'valid_3', 'status_3', 'processed_url_3']
    for col in cols:
        if col not in links_df.columns:
            links_df[col] = None

    # get final validity, prefer the last valid url
    if len(links_df) > 0:
        links_df['end_processed_valid'] = links_df['valid_3'].fillna(
            links_df['valid_2'].fillna(
                links_df['valid_1']
            )
        )

        links_df['end_processed_status'] = links_df['status_3'].fillna(
            links_df['status_2'].fillna(
                links_df['status_1']
            )
        )
        
        links_df['end_processed_url'] = links_df['processed_url_3'].fillna(
            links_df['processed_url_2'].fillna(
                links_df['processed_url_1']
            )
        )
        
        # print('l', links_df)
        # get redirects for 3xx errors
        error3xxs = links_df[(links_df['status_1']>=300)&(links_df['status_1']<400)|
                             (links_df['status_2']>=300)&(links_df['status_2']<400)|
                             (links_df['status_3']>=300)&(links_df['status_3']<400)
                             ][['end_processed_url']].drop_duplicates().copy()

        rd_results = await validate_urls_main(error3xxs['end_processed_url'], retries=100000000, allow_redirects=True)
        rd_df = pd.DataFrame([[x[0], *x[1]] for x in rd_results],
                             columns=['end_processed_url', 'valid_rd', 'status_rd', 'redirected_url']
                             ).drop_duplicates('end_processed_url')
        # print('rd', rd_df)
                             
        links_df = links_df.merge(rd_df, on='end_processed_url', how='left').drop_duplicates()

        #  rerun for 429 errors
        error429s = links_df[(links_df['status_1']==429)|
                                (links_df['status_2']==429)|
                                (links_df['status_3']==429)|
                                (links_df['status_rd']==429)
                                ][links_df.columns[:chop_index]].drop_duplicates().copy()
        links_df = links_df[(links_df['status_1']!=429)&
                            (links_df['status_2']!=429)&
                            (links_df['status_3']!=429)&
                            (links_df['status_rd']!=429)
                            ].drop_duplicates()

        if len(error429s) > 0:
            if retry_count < max_retries:
                delay = backoff_factor ** retry_count
                print(f"429 errors: {len(error429s)}. Retrying after {delay} seconds...")
                await asyncio.sleep(delay)
                # Retry recursively with increased retry_count
                new_links = await process_links(error429s.copy(), retry_count=retry_count + 1, max_retries=max_retries, backoff_factor=backoff_factor)
                links_df = pd.concat([links_df, new_links]).drop_duplicates()
            else:
                print(f"Max retries reached for {len(error429s)} URLs. Skipping...")
                error429s_extra = links_df[(links_df['status_1']==429)|
                                (links_df['status_2']==429)|
                                (links_df['status_3']==429)|
                                (links_df['status_rd']==429)
                                ][links_df.columns[:chop_index]].drop_duplicates().copy()
                error429s = pd.concat([error429s, error429s_extra])
                links_df = links_df[(links_df['status_1']!=429)&
                            (links_df['status_2']!=429)&
                            (links_df['status_3']!=429)&
                            (links_df['status_rd']!=429)
                            ].drop_duplicates()
                # print(f"429 errors: {len(l1429)} -> {len(error429s)}")
                # save 429s
                if len(error429s) > 0:
                    print("Saving 429s")
                    error429s.to_hdf('data/429s.h5', key='df', mode='a')


        links_df['final_valid'] = links_df['valid_rd'].fillna(links_df['end_processed_valid'])
        links_df['final_status'] = links_df['status_rd'].fillna(links_df['end_processed_status'])
        links_df['final_url'] = links_df['redirected_url'].fillna(links_df['end_processed_url'])

        print("Saving cache. Total size: ", len(RD_URL_CACHE_P[True]) + len(RD_URL_CACHE_P[False])
                + len(RD_URL_CACHE_TP))
        with open('.temp/rd_url_cache_p.pkl', 'wb') as f:
            pickle.dump(RD_URL_CACHE_P, f)
        with open('.temp/rd_url_cache_tp.pkl', 'wb') as f:
            pickle.dump(RD_URL_CACHE_TP, f)

        return links_df.drop_duplicates().reset_index(drop=True)
    else:
        return links_df.drop_duplicates().reset_index(drop=True)





In [3]:
# RD_URL_CACHE_P = {True: {}, False: {}}
# RD_URL_CACHE_TP = {}

# print("Saving cache. Total size: ", len(RD_URL_CACHE_P[True]) + len(RD_URL_CACHE_P[False])
#         + len(RD_URL_CACHE_TP))
# with open('.temp/rd_url_cache_p.pkl', 'wb') as f:
#     pickle.dump(RD_URL_CACHE_P, f)
# with open('.temp/rd_url_cache_tp.pkl', 'wb') as f:
#     pickle.dump(RD_URL_CACHE_TP, f)


Saving cache. Total size:  0


In [2]:
# For example:
posts = pd.read_hdf('data/posts.h5').reset_index(drop=True)
ptest = posts.iloc[:1000].copy()  # limit for testing

# Run the async function
links_df = await get_links_df(ptest, column='body')

/tmp/ipykernel_805388/2514332107.py:308: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  links_df['valid_2'].fillna(
/tmp/ipykernel_805388/2514332107.py:307: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  links_df['end_processed_valid'] = links_df['valid_3'].fillna(


429 errors: 1. Retrying after 1 seconds...
429 errors: 1. Retrying after 3 seconds...


/tmp/ipykernel_805388/2514332107.py:380: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  links_df['final_valid'] = links_df['valid_rd'].fillna(links_df['end_processed_valid'])
/tmp/ipykernel_805388/2514332107.py:381: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  links_df['final_status'] = links_df['status_rd'].fillna(links_df['end_processed_status'])


Saving cache. Total size:  668243


/tmp/ipykernel_805388/2514332107.py:380: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  links_df['final_valid'] = links_df['valid_rd'].fillna(links_df['end_processed_valid'])
/tmp/ipykernel_805388/2514332107.py:381: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  links_df['final_status'] = links_df['status_rd'].fillna(links_df['end_processed_status'])


Saving cache. Total size:  668243


/tmp/ipykernel_805388/2514332107.py:359: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  links_df = pd.concat([links_df, new_links]).drop_duplicates()
/tmp/ipykernel_805388/2514332107.py:380: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  links_df['final_valid'] = links_df['valid_rd'].fillna(links_df['end_processed_valid'])


Saving cache. Total size:  668243


In [5]:
for n, x in enumerate(['valid_1', 'valid_2', 'valid_3']):
    for m, y in enumerate(['valid_1', 'valid_2', 'valid_3']):
        if m > n:
            print(x, y)
            display(links_df[links_df[x] & links_df[y]])

valid_1 valid_2


,id,subreddit_id,title,body,url,author_id,nsfw,score,upvote_ratio,distinguished,...,processed_url_3,end_processed_valid,end_processed_status,end_processed_url,valid_rd,status_rd,redirected_url,final_valid,final_status,final_url


valid_1 valid_3


,id,subreddit_id,title,body,url,author_id,nsfw,score,upvote_ratio,distinguished,...,processed_url_3,end_processed_valid,end_processed_status,end_processed_url,valid_rd,status_rd,redirected_url,final_valid,final_status,final_url


valid_2 valid_3


,id,subreddit_id,title,body,url,author_id,nsfw,score,upvote_ratio,distinguished,...,processed_url_3,end_processed_valid,end_processed_status,end_processed_url,valid_rd,status_rd,redirected_url,final_valid,final_status,final_url


In [2]:
posts = pd.read_hdf('data/posts.h5').reset_index(drop=True)

In [3]:
#  run on titles

outcols = ['id', 'extracted_url', 'valid_1', 'status_1',
       'processed_url_1', 'reextracted_url', 'valid_2', 'status_2',
       'processed_url_2', 'valid_3', 'status_3', 'processed_url_3',
       'end_processed_valid', 'end_processed_url',
       'valid_rd', 'status_rd', 'redirected_url', 'final_valid',
       'final_status', 'final_url']

size = len(posts)//10
titlelinks = []
for batch in range(0, len(posts), size):
    print(f"Batch {batch//size + 1}", size)
    try:
        titlelinks.append(pd.read_hdf(f'data/titlelinks_batches.h5', key=f'/batch_{batch//size + 1}'))
    except (FileNotFoundError, KeyError) as ex:
        print(ex)
        links_df = await get_links_df(posts.iloc[batch:batch+size].copy(), column='title')
        if len(links_df) > 0:
            print('Saving...')
            links_df[outcols].to_hdf(f'data/titlelinks_batches.h5', key=f'/batch_{batch//size + 1}', mode='a')
            titlelinks.append(links_df[outcols])

titlelinks = pd.concat(titlelinks).reset_index(drop=True)
if os.path.exists('data/titlelinks.h5'):
    os.remove('data/titlelinks.h5')
titlelinks.to_hdf('data/titlelinks.h5', key='df', mode='w')

Batch 1 33589
File data/titlelinks_batches.h5 does not exist


/tmp/ipykernel_1226366/2514332107.py:308: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  links_df['valid_2'].fillna(
/tmp/ipykernel_1226366/2514332107.py:307: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  links_df['end_processed_valid'] = links_df['valid_3'].fillna(
/tmp/ipykernel_1226366/2514332107.py:380: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasti

Saving cache. Total size:  3100797
Saving...
Batch 2 33589
'No object named /batch_2 in the file'


/tmp/ipykernel_1226366/2031526577.py:21: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['id', 'extracted_url', 'processed_url_1', 'reextracted_url', 'valid_2',
       'processed_url_2', 'valid_3', 'processed_url_3', 'end_processed_url',
       'valid_rd', 'redirected_url', 'final_url'],
      dtype='object')]

  links_df[outcols].to_hdf(f'data/titlelinks_batches.h5', key=f'/batch_{batch//size + 1}', mode='a')
/tmp/ipykernel_1226366/2514332107.py:308: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  links_df['valid_2'].fillna(
/tmp/ipykernel_1226366/2514332107.py:307: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill 

Saving cache. Total size:  3100797
Saving...
Batch 3 33589
'No object named /batch_3 in the file'


...

/tmp/ipykernel_1226366/2031526577.py:21: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['id', 'extracted_url', 'processed_url_1', 'reextracted_url', 'valid_2',
       'processed_url_2', 'valid_3', 'processed_url_3', 'end_processed_url',
       'valid_rd', 'redirected_url', 'final_url'],
      dtype='object')]

  links_df[outcols].to_hdf(f'data/titlelinks_batches.h5', key=f'/batch_{batch//size + 1}', mode='a')
/tmp/ipykernel_1226366/2514332107.py:308: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  links_df['valid_2'].fillna(
/tmp/ipykernel_1226366/2514332107.py:307: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill 

Saving cache. Total size:  3100797
Saving...
Batch 10 33589
'No object named /batch_10 in the file'


/tmp/ipykernel_1226366/2031526577.py:21: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['id', 'extracted_url', 'processed_url_1', 'reextracted_url', 'valid_2',
       'processed_url_2', 'valid_3', 'processed_url_3', 'end_processed_url',
       'valid_rd', 'redirected_url', 'final_url'],
      dtype='object')]

  links_df[outcols].to_hdf(f'data/titlelinks_batches.h5', key=f'/batch_{batch//size + 1}', mode='a')
/tmp/ipykernel_1226366/2514332107.py:308: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  links_df['valid_2'].fillna(
/tmp/ipykernel_1226366/2514332107.py:307: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill 

Saving cache. Total size:  3100797
Saving...
Batch 11 33589
'No object named /batch_11 in the file'


/tmp/ipykernel_1226366/2031526577.py:21: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['id', 'extracted_url', 'processed_url_1', 'reextracted_url', 'valid_2',
       'processed_url_2', 'valid_3', 'processed_url_3', 'end_processed_url',
       'valid_rd', 'redirected_url', 'final_url'],
      dtype='object')]

  links_df[outcols].to_hdf(f'data/titlelinks_batches.h5', key=f'/batch_{batch//size + 1}', mode='a')
/tmp/ipykernel_1226366/2031526577.py:27: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->Index(['id', 'extracted_url', 'processed_url_1', 'reextracted_url', 'valid_2',
       'processed_url_2', 'valid_3', 'processed_url_3', 'end_processed_url',
       'valid_rd', 'redirected_url', 'final_url'],
      dtype='object')]

  tit

In [ ]:
# problem IDs t3_udac4z, t3_ud2feb

In [ ]:
posts = pd.read_hdf('data/posts.h5').reset_index(drop=True)

In [6]:
# run on bodies
outcols = ['id', 'extracted_url', 'valid_1', 'status_1',
       'processed_url_1', 'reextracted_url', 'valid_2', 'status_2',
       'processed_url_2', 'valid_3', 'status_3', 'processed_url_3',
       'end_processed_valid', 'end_processed_url',
       'valid_rd', 'status_rd', 'redirected_url', 'final_valid',
       'final_status', 'final_url']
size = len(posts)//10

bodylinks = []
for batch in range(0, len(posts), size):
   print(f"Batch {batch//size + 1}", size)
   try:
      bodylinks.append(pd.read_hdf(f'data/bodylinks_batches.h5', key=f'/batch_{batch//size + 1}'))
   except (FileNotFoundError, KeyError) as ex:
      print(ex)
      links_df = await get_links_df(posts.iloc[batch:batch+size].copy(), column='body')
      if len(links_df) > 0:
         print('saving')
         links_df[outcols].to_hdf(f'data/bodylinks_batches.h5', key=f'/batch_{batch//size + 1}', mode='a')
         bodylinks.append(links_df[outcols])

bodylinks = pd.concat(bodylinks).reset_index(drop=True)
if os.path.exists('data/bodylinks.h5'):
   os.remove('data/bodylinks.h5')
bodylinks.to_hdf('data/bodylinks.h5', key='df', mode='w')

Batch 1 33589
File data/bodylinks_batches.h5 does not exist
Invalid IPv6 URL
Invalid IPv6 URL
'Google have some books archived for free' does not appear to be an IPv4 or IPv6 address
Invalid IPv6 URL
Invalid IPv6 URL
Invalid IPv6 URL
Invalid IPv6 URL
Invalid IPv6 URL
Invalid IPv6 URL
Invalid IPv6 URL
Invalid IPv6 URL
Invalid IPv6 URL
Invalid IPv6 URL
Invalid IPv6 URL
Invalid IPv6 URL
Invalid IPv6 URL
Invalid IPv6 URL
Invalid IPv6 URL
Invalid IPv6 URL
'Cisco' does not appear to be an IPv4 or IPv6 address


/tmp/ipykernel_1226366/2514332107.py:308: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  links_df['valid_2'].fillna(
/tmp/ipykernel_1226366/2514332107.py:307: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  links_df['end_processed_valid'] = links_df['valid_3'].fillna(
/tmp/ipykernel_1226366/2514332107.py:380: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasti

Saving cache. Total size:  3100801
saving
Batch 2 33589
'No object named /batch_2 in the file'


/tmp/ipykernel_1226366/2336260079.py:20: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['id', 'extracted_url', 'processed_url_1', 'reextracted_url', 'valid_2',
       'processed_url_2', 'valid_3', 'processed_url_3', 'end_processed_url',
       'valid_rd', 'redirected_url', 'final_url'],
      dtype='object')]

  links_df[outcols].to_hdf(f'data/bodylinks_batches.h5', key=f'/batch_{batch//size + 1}', mode='a')


'Horse Sacrifice' does not appear to be an IPv4 or IPv6 address
Invalid IPv6 URL
Invalid IPv6 URL
Invalid IPv6 URL
Invalid IPv6 URL
Invalid IPv6 URL


...

/tmp/ipykernel_1226366/2336260079.py:20: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['id', 'extracted_url', 'processed_url_1', 'reextracted_url', 'valid_2',
       'processed_url_2', 'valid_3', 'processed_url_3', 'end_processed_url',
       'valid_rd', 'redirected_url', 'final_url'],
      dtype='object')]

  links_df[outcols].to_hdf(f'data/bodylinks_batches.h5', key=f'/batch_{batch//size + 1}', mode='a')
/tmp/ipykernel_1226366/2514332107.py:308: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  links_df['valid_2'].fillna(
/tmp/ipykernel_1226366/2514332107.py:307: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill i

Saving cache. Total size:  3100850
saving
Batch 11 33589
'No object named /batch_11 in the file'
Saving cache. Total size:  3100850


/tmp/ipykernel_1226366/2336260079.py:20: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['id', 'extracted_url', 'processed_url_1', 'reextracted_url', 'valid_2',
       'processed_url_2', 'valid_3', 'processed_url_3', 'end_processed_url',
       'valid_rd', 'redirected_url', 'final_url'],
      dtype='object')]

  links_df[outcols].to_hdf(f'data/bodylinks_batches.h5', key=f'/batch_{batch//size + 1}', mode='a')
/tmp/ipykernel_1226366/2514332107.py:308: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  links_df['valid_2'].fillna(
/tmp/ipykernel_1226366/2514332107.py:307: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill i

saving


/tmp/ipykernel_1226366/2336260079.py:20: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['id', 'extracted_url', 'processed_url_1', 'reextracted_url', 'valid_2',
       'processed_url_2', 'valid_3', 'processed_url_3', 'end_processed_url',
       'valid_rd', 'status_rd', 'redirected_url', 'final_url'],
      dtype='object')]

  links_df[outcols].to_hdf(f'data/bodylinks_batches.h5', key=f'/batch_{batch//size + 1}', mode='a')
/tmp/ipykernel_1226366/2336260079.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  bodylinks = pd.concat(bodylinks).reset_index(drop=True)
/tmp/ipykernel_1226366/2336260079.py:26: Perfor

In [2]:
comments = pd.concat([pd.read_hdf(f'data/comments_{x}.h5') for x in range(1,5)]).reset_index(drop=True)

In [ ]:
# run on comments
outcols = ['id', 'extracted_url', 'valid_1', 'status_1',
         'processed_url_1', 'reextracted_url', 'valid_2', 'status_2',
         'processed_url_2', 'valid_3', 'status_3', 'processed_url_3',
         'end_processed_valid', 'end_processed_url',
         'valid_rd', 'status_rd', 'redirected_url', 'final_valid',
         'final_status', 'final_url']
size = len(comments)//100

commentlinks = []
for batch in range(0, len(comments), size):
    print(f"Batch {batch//size + 1}", size)
    try:
        commentlinks.append(pd.read_hdf(f'data/commentlinks_batches.h5', key=f'/batch_{batch//size + 1}'))
    except (FileNotFoundError, KeyError) as ex:
        print(ex)
        while True:
            try:
                links_df = await get_links_df(comments.iloc[batch:batch+size].copy(), column='body')
                MAX_CONCURRENCY = min(200, int(MAX_CONCURRENCY*(2**0.5)))
                break
            except MemoryError as ex:
                print(ex)
                MAX_CONCURRENCY = MAX_CONCURRENCY//2
        if len(links_df) > 0:
            print('saving')
            links_df[outcols].to_hdf(f'data/commentlinks_batches.h5', key=f'/batch_{batch//size + 1}', mode='a')
            commentlinks.append(links_df[outcols])

commentlinks = pd.concat(commentlinks).reset_index(drop=True)
if os.path.exists('data/commentlinks.h5'):
    os.remove('data/commentlinks.h5')
commentlinks.to_hdf('data/commentlinks.h5', key='df', mode='w')

Batch 1 102643
Batch 2 102643
Batch 3 102643
Batch 4 102643
Batch 5 102643
Batch 6 102643
Batch 7 102643
Batch 8 102643
Batch 9 102643
Batch 10 102643
Batch 11 102643
Batch 12 102643
Batch 13 102643
Batch 14 102643
Batch 15 102643
Batch 16 102643
Batch 17 102643
Batch 18 102643
Batch 19 102643
Batch 20 102643
Batch 21 102643
Batch 22 102643
Batch 23 102643
Batch 24 102643
Batch 25 102643
'No object named /batch_25 in the file'
Invalid IPv6 URL

label empty or too long
Cannot connect to host en.www.wikipedia.org:443 ssl:default [Name or service not known]
Cannot connect to host www.en.wikipedia.org:443 ssl:default [Name or service not known]


Cannot connect to host www.en.wikipedia.org:443 ssl:default [Name or service not known]
Cannot connect to host www.en.wikipedia.org:443 ssl:default [Name or service not known]
Cannot connect to host en.www.wikipedia.org:443 ssl:default [Name or service not known]
Cannot connect to host en.www.wikipedia.org:443 ssl:default [Name or service not know

/tmp/ipykernel_996941/2514332107.py:308: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  links_df['valid_2'].fillna(
/tmp/ipykernel_996941/2514332107.py:307: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  links_df['end_processed_valid'] = links_df['valid_3'].fillna(


429 errors: 2665. Retrying after 1 seconds...


/tmp/ipykernel_996941/2514332107.py:308: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  links_df['valid_2'].fillna(
/tmp/ipykernel_996941/2514332107.py:307: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  links_df['end_processed_valid'] = links_df['valid_3'].fillna(


429 errors: 184. Retrying after 3 seconds...


...

/tmp/ipykernel_996941/2514332107.py:380: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  links_df['final_valid'] = links_df['valid_rd'].fillna(links_df['end_processed_valid'])


Saving cache. Total size:  1516321
saving
Batch 29 102643
'No object named /batch_29 in the file'


/tmp/ipykernel_996941/3146937327.py:27: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['id', 'extracted_url', 'processed_url_1', 'reextracted_url', 'valid_2',
       'processed_url_2', 'valid_3', 'processed_url_3', 'end_processed_url',
       'valid_rd', 'redirected_url', 'final_url'],
      dtype='object')]

  links_df[outcols].to_hdf(f'data/commentlinks_batches.h5', key=f'/batch_{batch//size + 1}', mode='a')


Invalid IPv6 URL
Invalid IPv6 URL
Invalid IPv6 URL
'this guy' does not appear to be an IPv4 or IPv6 address

label empty or too long
label empty or too long

Cannot connect to host www.en.wikipedia.org:443 ssl:default [Name or service not known]
label empty or too long
label empty or too long
label empty or too long
Cannot connect to host www.en.wikipedia.org:443 ssl:default [Name or service not known]


/tmp/ipykernel_996941/2514332107.py:308: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  links_df['valid_2'].fillna(
/tmp/ipykernel_996941/2514332107.py:307: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  links_df['end_processed_valid'] = links_df['valid_3'].fillna(


In [2]:
commentlinks = pd.concat([pd.read_hdf(f'data/commentlinks_batches.h5', key=f'/batch_{x}') for x in range(1,5)]).reset_index(drop=True)